## This compares to DoubleBracketIteration whenever possible

Note that we are not using the reduced group commutator so just 4 steps of the iteration using circuit representations uses quite a lot of time (several seconds)

In [1]:
import inspect
import sys
sys.path.append("../../tests")
from test_models_dbi import *
def print_function_source_code( func ):
    out = inspect.getsourcelines(func)  
    from functools import reduce
    print(reduce(str.__add__, out[0]))
import qibo
backend = qibo.backends.construct_backend("numpy")
qibo.set_backend("numpy")
nqubits = 3

[Qibo 0.2.7|INFO|2024-04-30 09:44:18]: Using numpy backend on /CPU:0


In [2]:
r=0.1
mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
h_x = SymbolicHamiltonian(
        symbols.X(0)
    + symbols.Z(0) * symbols.X(1)
    + symbols.Y(2)
    + symbols.Y(1) * symbols.Y(2),
nqubits=3,
)
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits=3)
h_input = h_x + d_0

dbi = DoubleBracketIteration(deepcopy(h_input.dense))

evolution_oracle = EvolutionOracle(h_input, "ZX", mode_evolution_oracle)   
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))

evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
   mode_evolution_oracle)

from numpy.linalg import norm

for _ in range(4):
    dbi(r, d = d_0.dense.matrix, mode = DoubleBracketGeneratorType.group_commutator )
    gci(r, diagonal_association= evolution_oracle_diagonal_target )

k_r = dbi.h.matrix
j_r = gci.h.matrix




[Qibo 0.2.7|WARNING|2024-04-30 09:44:18]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-30 09:44:18]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [3]:
u = gci.iterated_hamiltonian_evolution_oracle.get_composed_circuit()

In [4]:
h0 = h_input.dense.matrix
h_end = u.conj().T @ h0 @ u

AttributeError: 'Circuit' object has no attribute 'conj'

In [ ]:
norm(h_end - dbi.h.matrix)

In [ ]:
gci.input_hamiltonian_evolution_oracle.eps_trottersuzuki

In [ ]:
norm(h_end - dbi.h.matrix)/gci.input_hamiltonian_evolution_oracle.eps_trottersuzuki

We conclude that this is fine. Feel free to change to the numerical mode where the error will start piling up merely from the floating point operations and will build up from the double digit precision.